In [17]:
# 필요한 모듈 임포트
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    FewShotChatMessagePromptTemplate,
)
from langchain.memory import ChatMessageHistory
from langchain_openai import ChatOpenAI

# 1. 채팅 기록을 저장할 메모리 생성
#    대화 기록을 저장하고 불러오기 위해 ChatMessageHistory 객체 사용!
history = ChatMessageHistory()

# 2. OpenAI 모델 초기화
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 3. Few-shot 예제 설정
examples = [
    {"human": "The title of the movie is 'Top Gun'", "ai": "🛩️👨‍✈️🔥"},
    {"human": "The title of the movie is 'The Godfather'", "ai": "👨‍👨‍👦🔫🍝"},
    {"human": "The title of the movie is 'Jaws'", "ai": "🦈🏊‍♂️😱"},
]

# Few-shot 예제를 템플릿으로 변환
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{human}"),  # 사람이 입력한 메시지
        ("ai", "{ai}"),  # AI가 응답한 메시지
    ]
)

# Few-shot Chat Message Prompt Template 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 4. 체인에 사용할 프롬프트 설정
#    대화 기록과 함께 작동할 수 있도록 MessagesPlaceholder를 추가
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="영화 제목을 받으면 딱 3개의 이모지로만 응답하세요."
        ),  # 기본 시스템 지침
        few_shot_prompt,  # Few-shot 예제
        MessagesPlaceholder(variable_name="messages"),  # 메모리를 연결할 자리
    ]
)

# 프롬프트와 OpenAI 모델을 연결하여 체인을 생성
chain = prompt | chat

# 5. 첫 번째 영화 질문
# HumanMessage를 생성하여 체인에 전달합니다.
first_message = HumanMessage(content="The title of the movie is Matrix")

# ##### 핵심: 체인 호출 시 메모리(history.messages)를 포함. #####
# 체인에 전달할 메시지 리스트에 기존 대화 기록(history.messages)과 새 메시지(first_message)를 모두 포함.
first_response = chain.invoke({"messages": history.messages + [first_message]})

# ##### 메모리 업데이트 #####
# 대화 기록에 첫 번째 질문과 AI 응답을 저장.
history.add_message(first_message)
history.add_message(first_response)

# 첫 번째 질문에 대한 응답 출력
print("첫 번째 영화:", first_response.content)

# 6. 두 번째 영화 질문
# 새로운 HumanMessage를 생성
second_message = HumanMessage(content="The title of the movie is Titanic")

# ##### 이전 대화 기록(history.messages)을 포함 #####
# 두 번째 질문에도 기존 대화 기록을 포함하여 체인이 맥락을 이해할 수 있도록
second_response = chain.invoke({"messages": history.messages + [second_message]})

# ##### 메모리 업데이트 #####
# 대화 기록에 두 번째 질문과 AI 응답을 저장
history.add_message(second_message)
history.add_message(second_response)

# 두 번째 질문에 대한 응답 출력
print("두 번째 영화:", second_response.content)

# 7. 메모리에서 첫 번째 영화 질문 및 응답 확인
# ##### 메모리 활용 #####
# 메모리(history.messages)를 사용하여 첫 번째 질문과 AI 응답을 추출
messages = history.messages
first_movie = (
    messages[0].content.split("is ")[-1].strip()
)  # 첫 번째 영화 제목 추출 ("Matrix")
first_emoji = messages[1].content  # 첫 번째 질문에 대한 AI 응답 추출

# 첫 번째 영화 질문과 응답 출력
print(f"첫 번째로 물어본 영화: {first_movie} {first_emoji}")

첫 번째 영화: 💊🕶️💻
두 번째 영화: 🚢💔🌊
첫 번째로 물어본 영화: Matrix 💊🕶️💻
